# Camera Alignment

This notebook aligns the 2 cameras on the telescope (Finder Camera and main OTA camera). Information on the finder camera can be found at _/telescope-control/finder-camera/finder-camera-information.md_

## Questions:

### Can the All Sky camera be used to put a start in the telescopes OTA field of view?

* The ASI482MC's theoretical FOV is 23.91 x 13.45 arcmin (see _astrophotography_camera_exposure_times.ipynb_)
* The ASI120MC-S 150 All Sky's theoretical FOV per pixel is 422 arcseconds or approximately 7 arcmin (see _astrophotography_camera_exposure_times.ipynb_)

Based on the above information it should be possible to get whatever a singe pixel points at in the camera.

### What is the best exposure for the finder camera?

* The ASI120MC-S's All sky max exposure is 176 seconds before images start to blur (see _astrophotography_camera_exposure_times.ipynb_)
* User needs feedback something is happening if it takes over a second. 
* Longer the exposure time the more we can draw light from the sky.
* The finder cameras exposure time is independent of the use off tracking.

On movement ramp the exposure time up. This would make stars to appear dimmer but to become sharper and exceed normal human sight. This way you get feedback the telescope is moving based on the movement of terrestrial things in the view. But once the motion stops the stars become more pronounced. This is similar to the painters algorithm in CAD. Things closer are drawn first.

### What is the best gain for the finder camera?

* The primary point of gain is to expose the terrestrial objects at night as a reference point. So we want to set the gain based on the fastest exposure.
* The gain should auto adjust based on the darkness of the night. This will allow for twilight viewing and for the strength of the moon if present.

## Setup Notebook

Make sure everything is up to date.


In [1]:
!python3 --version
!python3 -m pip install --upgrade pip
%pip install pandas
%pip install ipywidgets

Python 3.10.4
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Imports Used

In [2]:
import pandas as pd
from workflow_logger import WorkflowLogger
import numpy as np
import scipy.misc
import asi

## Terrestrial Daylight Alignment

The idea is that I can use a daylight distant object like a power line connection to align the 2 cameras. We will want the zoom level to be high enough to ensure we can get the exact pixel position.

In [4]:
workflow_log : pd.DataFrame = WorkflowLogger().get_workflow_log()

# List the connected camera's
cameras = asi.getConnectedCameras()
print("Connected camera's:")

for camera_number in cameras:
    print(cameras[camera_number]['Name'])

print(dir(asi))
print(dir(asi.Camera))

In [ ]:
import asi
import cv2
from typing import Optional

class AlignmentFinderCamera(object):
    _video = None
    _exposure = 2000000
    _offset_cross = (-12,-8)
    _gain = 0
    last_image: Optional[bytes] = None

    def __init__(self):
        self._video = asi.Camera(0)
        self._video.setCaptureFrameFormat(1280, 960, 1, "RGB24")
        self._video.setControlValueManual("ASI_GAIN", self._gain)
        self._video.setControlValueManual("ASI_WB_R", 55)
        self._video.setControlValueManual("ASI_WB_B", 77)

    def __del__(self):
        del self._video

    def get_frame(self) -> bytes:
        self.last_image, bin, success = self._video.grab(self._exposure)
        b,g,r = cv2.split(self.last_image)
        corrected_image = cv2.merge ( (r, g, b) )
        #print(self.last_image.shape)
        png = corrected_image[80-self._offset_cross[0]:880-self._offset_cross[0],240-self._offset_cross[1]:1040-self._offset_cross[1]]
        #print(png.shape)
        #ret, png = cv2.imencode('.png', self.last_image)

        cross = cv2.imread('overlays/alignment-cross-hair.png')
        #print(cross.shape)
        #overlay = cv2.imencode('.png',cv2.imread('cross-hair.png'))
        added_image = cv2.addWeighted(png,1,cross,1,0,dtype = cv2.CV_32F)
        
        ret, jpeg = cv2.imencode('.jpg', added_image)
        return jpeg.tobytes()

class AlignmentOtaCamera(object):
    _video = None
    _exposure = 2000000
    _offset_cross = (0,0)
    _gain = 0
    last_image: Optional[bytes] = None

    def __init__(self):
        self._video = asi.Camera(0)
        self._video.setCaptureFrameFormat(1920, 1080, 1, "RGB24")
        self._video.setControlValueManual("ASI_GAIN", self._gain)
        self._video.setControlValueManual("ASI_WB_R", 55)
        self._video.setControlValueManual("ASI_WB_B", 77)

    def __del__(self):
        del self._video

    def get_frame(self) -> bytes:
        self.last_image, bin, success = self._video.grab(self._exposure)
        b,g,r = cv2.split(self.last_image)
        corrected_image = cv2.merge ( (r, g, b) )
        #print(self.last_image.shape)
        png = corrected_image[80-self._offset_cross[0]:880-self._offset_cross[0],240-self._offset_cross[1]:1040-self._offset_cross[1]]
        #print(png.shape)
        #ret, png = cv2.imencode('.png', self.last_image)

        cross = cv2.imread('overlays/alignment-cross-hair.png')
        #print(cross.shape)
        #overlay = cv2.imencode('.png',cv2.imread('cross-hair.png'))
        added_image = cv2.addWeighted(png,1,cross,1,0,dtype = cv2.CV_32F)
        
        ret, jpeg = cv2.imencode('.jpg', added_image)
        return jpeg.tobytes()



In [ ]:
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import threading

app = FastAPI()

def run():
    uvicorn.run(app)
      
def start_api():
    _api_thread = threading.Thread(target=run)
    _api_thread.start()
    
def create_alignment_finder_stream(): 
    alignment_finder_camera: AlignmentFinderCamera  = AlignmentFinderCamera()    
    while True:
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + bytearray(alignment_finder_camera.get_frame()) + b'\r\n\r\n')

def create_alignment_ota_stream(): 
    alignment_ota_camera: AlignmentOtaCamera  = AlignmentOtaCamera()    
    while True:
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + bytearray(alignment_ota_camera.get_frame()) + b'\r\n\r\n')
      
@app.get("/finder")
def get_alignment_finder_video_stream():
    return StreamingResponse(create_alignment_finder_stream(), media_type='multipart/x-mixed-replace; boundary=frame')
    
@app.get("/ota")
def create_alignment_ota_stream():
    return StreamingResponse(create_alignment_finder_stream(), media_type='multipart/x-mixed-replace; boundary=frame')

start_api()

### Step 1: Set the finder camera for daylight viewing

In [ ]:
%%HTML 
<img src="http://127.0.0.1:8000/finder" width=640 />

WorkflowLogger('Step 1: Set the finder camera for daylight viewing', workflow_log).display()

### Step 2: Set the OTA camera for daylight viewing. Display a cross-hair object in the center of the OTA camera.

In [ ]:
%%HTML 
<img src="http://127.0.0.1:8000/ota" width=640 />

WorkflowLogger('Step 2: Set the OTA camera for daylight viewing', workflow_log).display()

### Step 3: Point the telescopes OTA at any easy to identify object

In [ ]:
WorkflowLogger('Step 1', workflow_log).display()

### Step 4: Set the cross-hair on the finder camera to correspond with object

In [ ]:
WorkflowLogger('Step 1', workflow_log).display()

### Step 5: Point the finder scope at a different object, record the offset difference in the OTA (x and y)

In [ ]:
WorkflowLogger('Step 1', workflow_log).display()

### Step 6: Point the finder scope at a different object, record the offset difference in the OTA (x and y)

In [ ]:
WorkflowLogger('Step 1', workflow_log).display()

### Step 7: Point the finder scope at a different object, record the offset difference in the OTA (x and y)

In [ ]:
WorkflowLogger('Step 1', workflow_log).display()

### Step 8: Point the finder scope at a different object, record the offset difference in the OTA (x and y)

In [ ]:
WorkflowLogger('Step 1', workflow_log).display()

### Step 9: Point the finder scope at a different object, record the offset difference in the OTA (x and y)

In [ ]:
WorkflowLogger('Step 1', workflow_log).display()

### Step 10: Save alignment data and workflow results to the cloud.

In [5]:
WorkflowLogger('Step 1', workflow_log).display()

Textarea(value='', description='Comments:')

Button(description='Done', style=ButtonStyle())

Output()